In [ ]:
!pip install aioquic
!pip install nest_asyncio

In [35]:
import asyncio
import logging
import nest_asyncio
from aioquic.asyncio import connect
from aioquic.quic.configuration import QuicConfiguration

# Enable logging for debugging (optional)
logging.basicConfig(level=logging.DEBUG)

# Apply nested asyncio to avoid conflicts in Google Colab
nest_asyncio.apply()


async def check_congestion_control(server, port, congestion_control):
    """Attempts to connect using the specified congestion control algorithm."""
    config = QuicConfiguration(is_client=True)
    config.alpn_protocols = ["h3"]  # Explicitly set ALPN for HTTP/3
    config.congestion_control_algorithm = congestion_control

    try:
        async with connect(server, port, configuration=config):
            print(f"Server supports {congestion_control}")
            return True
    except Exception as e:
        print(f"Server does not support {congestion_control}: {e}")
        return False


async def main():
    server = "cloudflare-quic.com"
    port = 443
    congestion_controls = ["reno", "cubic", "bbr"]  # Add more if needed

    for cc in congestion_controls:
        await check_congestion_control(server, port, cc)


if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

Server supports reno
Server supports cubic
Server does not support bbr: Unknown congestion control algorithm: bbr
